In [1]:
import os
import time
import requests
import pandas as pd

In [ ]:

APP_TOKEN = os.getenv("SOCRATA_APP_TOKEN") 
BASE = "https://data.cityofchicago.org/resource/v6vf-nfxy.json"

In [3]:
def fetch_311(created_after="2025-10-01T00:00:00.000", limit=50000, max_pages=50):
    headers = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}
    all_rows = []
    offset = 0

    where = (
        f"created_date >= '{created_after}' "
        "AND sr_type != '311 INFORMATION ONLY CALL'"
    )

    select = ",".join([
        "sr_number", "sr_type", "status",
        "created_date", "completion_date",
        "street_address",
        "community_area", "ward", "police_district",
        "latitude", "longitude"
    ])

    for _ in range(max_pages):
        params = {
            "$select": select,
            "$where": where,
            "$limit": limit,
            "$offset": offset
        }
        r = requests.get(BASE, headers=headers, params=params, timeout=60)
        r.raise_for_status()
        rows = r.json()
        if not rows:
            break

        all_rows.extend(rows)
        offset += limit
        time.sleep(0.2)  # be polite

    return pd.DataFrame(all_rows)

In [4]:
df = fetch_311(created_after="2025-07-01T00:00:00.000")
print(df.shape)

HTTPError: 400 Client Error: Bad Request for url: https://data.cityofchicago.org/resource/v6vf-nfxy.json?%24select=sr_number%2Csr_type%2Cstatus%2Ccreated_date%2Ccompletion_date%2Cstreet_address%2Ccommunity_area%2Cward%2Cpolice_district%2Clatitude%2Clongitude&%24where=created_date+%3E%3D+%272025-07-01T00%3A00%3A00.000%27+AND+sr_type+%21%3D+%27311+INFORMATION+ONLY+CALL%27&%24limit=50000&%24offset=0